In [1]:
from typing import Tuple

import pandas

from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
country = 'canada'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    return df_mrd, df_covid_measures

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
# param_grid = {  
#     'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
#     'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
# }

model = Prophet().fit(df_train)

02:51:25 - cmdstanpy - INFO - Chain [1] start processing
02:51:25 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
def test_model(df_test: pandas.DataFrame, model: Prophet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [9]:
df_predicted, rmse = test_model(df_test, model)

In [10]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-04-01,6.921428,5.440729,8.695165,6.921428,6.921428,0.095213,0.095213,0.095213,0.095213,0.095213,0.095213,0.0,0.0,0.0,7.016642
1,2022-05-01,6.919881,5.382347,8.529390,6.919881,6.919881,0.081249,0.081249,0.081249,0.081249,0.081249,0.081249,0.0,0.0,0.0,7.001131
2,2022-06-01,6.918283,5.295254,8.528658,6.918283,6.918283,0.052664,0.052664,0.052664,0.052664,0.052664,0.052664,0.0,0.0,0.0,6.970947
3,2022-07-01,6.916736,5.407900,8.598271,6.916736,6.916736,0.038030,0.038030,0.038030,0.038030,0.038030,0.038030,0.0,0.0,0.0,6.954766
4,2022-08-01,6.915137,5.464360,8.389620,6.915137,6.915137,0.006462,0.006462,0.006462,0.006462,0.006462,0.006462,0.0,0.0,0.0,6.921599
5,2022-09-01,6.913539,5.323967,8.487116,6.913539,6.913539,-0.000525,-0.000525,-0.000525,-0.000525,-0.000525,-0.000525,0.0,0.0,0.0,6.913014
6,2022-10-01,6.911992,5.321855,8.474934,6.911992,6.911992,-0.008693,-0.008693,-0.008693,-0.008693,-0.008693,-0.008693,0.0,0.0,0.0,6.903299
7,2022-11-01,6.910393,5.322733,8.310976,6.909811,6.910922,-0.014040,-0.014040,-0.014040,-0.014040,-0.014040,-0.014040,0.0,0.0,0.0,6.896353
8,2022-12-01,6.908847,5.255186,8.441290,6.906688,6.910048,-0.014039,-0.014039,-0.014039,-0.014039,-0.014039,-0.014039,0.0,0.0,0.0,6.894808
9,2023-01-01,6.907248,5.360682,8.412478,6.904377,6.909495,-0.017476,-0.017476,-0.017476,-0.017476,-0.017476,-0.017476,0.0,0.0,0.0,6.889772


In [11]:
rmse

1.8502883975756614

In [12]:
with open(f'{country}_prophet_base_model.json', 'w') as f:
    f.write(model_to_json(model))

In [13]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [14]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

In [15]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,6.904206,5.415740,8.577039,6.904206,6.904206,0.096967,0.096967,0.096967,0.096967,0.096967,0.096967,0.0,0.0,0.0,7.001173
1,2023-04-01,6.902607,5.441462,8.507586,6.902607,6.902607,0.076616,0.076616,0.076616,0.076616,0.076616,0.076616,0.0,0.0,0.0,6.979223
2,2023-05-01,6.901060,5.424136,8.514531,6.901060,6.901060,0.047671,0.047671,0.047671,0.047671,0.047671,0.047671,0.0,0.0,0.0,6.948732
